
### The University of Melbourne, School of Computing and Information Systems
# COMP30027 Machine Learning, 2020 Semester 1

## Assignment 1: Naive Bayes Classifiers

###### Submission deadline: 7 pm, Monday 20 Apr 2020

**Student Name(s):**    `Alec Yu, Michael Jaworski`

**Student ID(s):**     `993433, `


This iPython notebook is a template which you will use for your Assignment 1 submission.

Marking will be applied on the four functions that are defined in this notebook, and to your responses to the questions at the end of this notebook (Submitted in a separate PDF file).

**NOTE: YOU SHOULD ADD YOUR RESULTS, DIAGRAMS AND IMAGES FROM YOUR OBSERVATIONS IN THIS FILE TO YOUR REPORT (the PDF file).**

You may change the prototypes of these functions, and you may write other functions, according to your requirements. We would appreciate it if the required functions were prominent/easy to find.

**Adding proper comments to your code is MANDATORY. **

In [1]:
import numpy as np
from collections import defaultdict
import re
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd

In [19]:
# Lists for all the datafiles and their types
nominal_files = ["breast-cancer-wisconsin", "mushroom", "lymphography"]
numeric_files = ["wdbc", "wine"]
ordinal_files = ["car", "nursery", "somerville"]
mixed_files = ["adult", "bank", "university"]

# use trainingtest" for naive bayes example in slides
file = "mushroom"
filename = "datasets/%s.data" % file
headerfile = "datasets/%s.h" % file

if(file in nominal_files):
    datatype = "nominal"
elif (file in ordinal_files):
    datatype = "ordinal"
elif (file in numeric_files):
    datatype = "numeric"
elif (file in mixed_files):
    datatype = "mixed"

In [20]:
# Read the datafile in with a separate header file for attributes
dataframe = pd.read_csv(filename, header=None)
header = open(headerfile, "r")
attributes = header.readline().split(",")
dataframe.columns = attributes

In [21]:
# This function should prepare the data by reading it from a file and converting it into a useful format for training and testing

def preprocess(dataframe, datatype): 
    # Create a copy of the original dataframe. Remove rows with ? in this copy
    df = dataframe

    # If it's mushroom data, drop the stalk-root column as it has too many missing values. Don't use this attribute
    if(filename == "datasets/mushroom.data"):
        df = df.drop(columns = ['stalk-root'])

    # Otherwise just remove the entries with a ? for now
    for index in df.index:
        if('?' in df.loc[index].values):
            df = df.drop(index)
    
    # Leave nominal, ordinal and numeric data as it is.
    
    # If the data is mixed, we need to discretise the data, as each mixed data set is a classification task using NB
    if(datatype == 'mixed'):
        df = discretiseNumeric(df)
        
    # Also, move CLASS column to the end
    temp = df.pop('CLASS')
    df['CLASS'] = temp
    
    return df

############################################################################################################################
# Choice is to use equal-frequency discretisation. More detailed than equal-width discretisation. Here, computing an 
# "Optimal" number of clusters for k means clustering for every numeric column would not be feasible. If choosing an 
# Arbitrary K clusters for every single discretisation, then it isn't as effective
# For equal-frequency, sometimes the attributes won't have many values, so migth be less than 5 bins. Thats okay though

def discretiseNumeric(df):
    for column in df.columns:
        if(df.dtypes[column] == "int64" or df.dtypes[column] == "float64"):
            values = df[column].values
            values.sort()
            
            # Check if the values are already "Discretised" (<, say, 8 unique values. This occurs for ratings 0-5 etc..).
            # If so, then no need to discretise the attribute
            uniqueValues = set(values)
            if(len(uniqueValues) <= 8):
                continue
            
            # Choose 5 equal-frequency bins, based on sorted values
            # Should every attribute be split into 5 bins?
            
            bin1 = values[(int)(len(values)/5)]
            bin2 = values[2 * (int)(len(values)/5)]
            bin3 = values[3 * (int)(len(values)/5)]
            bin4 = values[4 * (int)(len(values)/5)]
            bin5 = values[5 * (int)(len(values)/5)]
            
            # Now assign every value to be in a bin number
            for i in range(len(df[column].values)):
                if(df[column].values[i] <= bin1):
                    df[column].values[i] = 1
                elif(df[column].values[i] <= bin2):
                    df[column].values[i] = 2
                elif(df[column].values[i] <= bin3):
                    df[column].values[i] = 3
                elif(df[column].values[i] <= bin4):
                    df[column].values[i] = 4
                elif(df[column].values[i] <= bin5):
                    df[column].values[i] = 5
                
    return df

############################################################################################################################

df2 = dataframe.copy()
df2 = preprocess(df2, datatype)
df2

,cap-shape,cap-surface,cap-color,bruises?,odor,gill-attachment,gill-spacing,gill-size,gill-colour,stalk-shape,...,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat,CLASS
0,x,s,n,t,p,f,c,n,k,e,...,w,w,p,w,o,p,k,s,u,p
1,x,s,y,t,a,f,c,b,k,e,...,w,w,p,w,o,p,n,n,g,e
2,b,s,w,t,l,f,c,b,n,e,...,w,w,p,w,o,p,n,n,m,e
3,x,y,w,t,p,f,c,n,n,e,...,w,w,p,w,o,p,k,s,u,p
4,x,s,g,f,n,f,w,b,k,t,...,w,w,p,w,o,e,n,a,g,e
5,x,y,y,t,a,f,c,b,n,e,...,w,w,p,w,o,p,k,n,g,e
6,b,s,w,t,a,f,c,b,g,e,...,w,w,p,w,o,p,k,n,m,e
7,b,y,w,t,l,f,c,b,n,e,...,w,w,p,w,o,p,n,s,m,e
8,x,y,w,t,p,f,c,n,p,e,...,w,w,p,w,o,p,k,v,g,p
9,b,s,y,t,a,f,c,b,g,e,...,w,w,p,w,o,p,k,s,m,e


In [222]:
# This function should calculate prior probabilities and likelihoods from the training data and using
# them to build a naive Bayes model

def train(dataframe, datatype):
    
    if(datatype in ["nominal", "ordinal", "mixed"]):
        modeltype = 'NB'
    elif(datatype in ["numeric"]):
        modeltype = 'GNB'
    
    if(modeltype == 'NB'):
        trainNB(dataframe)
    elif(modeltype == 'GNB'):
        trainGNB(dataframe)
    return

In [274]:
def trainNB(dataframe): 
    
    # Count all the frequencies of each class label
    priors = dict()
    for index in dataframe.index:
        item = dataframe.loc[index]['CLASS']
        if(item in priors):
            priors[item] += 1
        else:
            priors[item] = 1
            
    # Sum the total to convert each class label to a probability
    total = sum(priors.values())
    
    # Each unique class label now has a prior probability as a value
    for key, value in priors.items():
        priors[key] = value/total
        
    # Once we have the priors, we need all the conditional probabilities
    # Use a list of dictionaries of dictionaries
    
    # We need these to iterate through them and count instances
    attributes = list(dataframe.columns)[:-1]
    uniqueClassLabels = list(set(dataframe['CLASS'].values))
    
    attributeDicts = list()
    
    # For each attribute create a dictionary with class labels as keys.
    # Values are another dictionary with unique values in that attribute,
    # Then count frequencies of uniqueattvalues given class label
    for attribute in attributes:
        dict1 = dict()
        uniqueAttributeValues = list(set(dataframe[attribute].values))
        for classLabel in uniqueClassLabels:
            dict2 = dict()
            dict1[classLabel] = dict2
            for attributeValue in uniqueAttributeValues:
                dict2[attributeValue] = dataframe[(dataframe[attribute] == attributeValue) & (dataframe['CLASS'] == classLabel)].shape[0]
        attributeDicts.append(dict1)
    
    # Once we have the frequencies, can just convert each to be probabilities
    for attributedict in attributeDicts:
        for classLabel, classLabelDict in attributedict.items():
            totalFreq = sum(classLabelDict.values())
            for attributeLevel, freq in classLabelDict.items():
                classLabelDict[attributeLevel] = freq / totalFreq

    # EACH DICTIONARY IN attributeDicts IS IN ORDER OF THE HEADER COLUMNS
    i = 0;
    for dictionary in attributeDicts:
        print(attributes[i])
        i += 1
        for classLabel, dict2 in dictionary.items():
            print(classLabel)
            print(dict2)
            print(sum(dict2.values()))
    
    return attributeDicts, priors

12.278732394366198
0.5379642302964683


In [318]:
def trainGNB(dataframe):
        
    # Count all the frequencies of each class label
    priors = dict()
    for index in dataframe.index:
        item = dataframe.loc[index]['CLASS']
        if(item in priors):
            priors[item] += 1
        else:
            priors[item] = 1
            
    # Sum the total to convert each class label to a probability
    total = sum(priors.values())
    
    # Each unique class label now has a prior probability as a value
    for key, value in priors.items():
        priors[key] = value/total
        
    # Just need to store mean, stdev for every attribute for each unique class label    
    meanDict = df2.groupby('CLASS').mean().to_dict()
    stdDict = df2.groupby('CLASS').std().to_dict()
    
    print(meanDict['feat11'])
    print(stdDict['feat11'])
    
    
    return meanDict, stdDict, priors

# Calculates probability density given a gaussian dist with mean and std 
def calculateDensity(mean, std, x):
    exponent = math.exp((-1.0/2) * (((x - mean) / std) ** 2))
    return (1 / (math.sqrt(2 * math.pi) * std)) * exponent

In [319]:
train(df2, datatype)

{1: 1.0620338983050848, 2: 1.0562816901408452, 3: 0.6827083333333334}
{1: 0.11648264131468114, 2: 0.20293680811654433, 3: 0.1144410948234819}


In [83]:
# This function should predict classes for new items in a test dataset (for the purposes of this assignment, you
# can re-use the training data as a test set

# Multiplying probs, need to take logs of probabilities
# Smoothe training probabilities
# Change format of the probabilities, and ways of accessing them
def predict():
    return

In [48]:
# This function should evaliate the prediction performance by comparing your model’s class outputs to ground
# truth labels

# CODE TO RUN M-CROSS VALIDATION
m = 5;
width = int(dataframe.shape[0] / m);
evaluations = []
for i in range(0,m):
    testSet = dataframe.index[i*width:(i+1)*width]
    trainSet = dataframe.drop(dataframe.index[i*width:(i+1)*width])
    # Call predict and evaluate using thse two dataframes

# Choose appropriate metrics
# Crude total accuracy, confusion matrix of truth vs classified %'s 
# USE SKLEARN.metrics.confusionmatrix
from sklearn.metrics import confusion_matrix

def evaluate(truthlist, predictions):
    labels = list(set(truthlist))
    confusion_matrix(truthlist, predictions, labels=labels)
    
    return


# Finds the most common class label and returns it. Use this to predict all instances as a baseline
def zeroR(df):
    classLabel = df['CLASS'].value_counts().idxmax();
    return classLabel

testTruth = []
testPred = []

## Questions 


If you are in a group of 1, you will respond to question (1), and **one** other of your choosing (two responses in total).

If you are in a group of 2, you will respond to question (1) and question (2), and **two** others of your choosing (four responses in total). 

A response to a question should take about 100–250 words, and make reference to the data wherever possible.

#### NOTE: you may develope codes or functions in respond to the question, but your formal answer should be added to a separate file.

### Q1
Try discretising the numeric attributes in these datasets and treating them as discrete variables in the na¨ıve Bayes classifier. You can use a discretisation method of your choice and group the numeric values into any number of levels (but around 3 to 5 levels would probably be a good starting point). Does discretizing the variables improve classification performance, compared to the Gaussian na¨ıve Bayes approach? Why or why not?

### Q2
Implement a baseline model (e.g., random or 0R) and compare the performance of the na¨ıve Bayes classifier to this baseline on multiple datasets. Discuss why the baseline performance varies across datasets, and to what extent the na¨ıve Bayes classifier improves on the baseline performance.

### Q3
Since it’s difficult to model the probabilities of ordinal data, ordinal attributes are often treated as either nominal variables or numeric variables. Compare these strategies on the ordinal datasets provided. Deterimine which approach gives higher classification accuracy and discuss why.

### Q4
Evaluating the model on the same data that we use to train the model is considered to be a major mistake in Machine Learning. Implement a hold–out or cross–validation evaluation strategy (you should implement this yourself and do not simply call existing implementations from `scikit-learn`). How does your estimate of effectiveness change, compared to testing on the training data? Explain why. (The result might surprise you!)

### Q5
Implement one of the advanced smoothing regimes (add-k, Good-Turing). Does changing the smoothing regime (or indeed, not smoothing at all) affect the effectiveness of the na¨ıve Bayes classifier? Explain why, or why not.

### Q6
The Gaussian na¨ıve Bayes classifier assumes that numeric attributes come from a Gaussian distribution. Is this assumption always true for the numeric attributes in these datasets? Identify some cases where the Gaussian assumption is violated and describe any evidence (or lack thereof) that this has some effect on the NB classifier’s predictions.